In [5]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [6]:
# Load the Excel file
data = pd.read_excel("NLP_Data.xlsx")


In [7]:
data.head()

,Description of the Grievance,Grievance Category,Grievance SubCategory
0,concerns regarding laboratory tests being bill...,Billing/Financial Dispute,Provider Claim Issues
1,dassatifaction with provider,Quality Of Service,Not Satisfied With Provider Services
2,Dissatisafaction with delay in care.,Access And Availability,Pharmacy
3,Dissatisafaction with Dental provider way of c...,Quality Of Service,Not Satisfied With Provider Services
4,Dissatisfaction for not being informed he had ...,Billing/Financial Dispute,Balance Billing


In [8]:
# Extracting columns
descriptions = data['Description of the Grievance']
categories = data['Grievance Category']


In [9]:
# Initialize NLTK and download necessary resources
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
wnl = WordNetLemmatizer()

# Text cleaning, tokenization, and lowercasing
cleaned_texts = []
cleaned_categories = []  # To store valid categories corresponding to cleaned texts
for description, category in zip(descriptions, categories):
    if isinstance(description, str) and isinstance(category, str):  # Check if both description and category are strings
        words = word_tokenize(description)
        words = [word.lower() for word in words if word.isalpha()]
        words = [word for word in words if word not in stop_words]
        words = [wnl.lemmatize(word) for word in words]
        cleaned_texts.append(" ".join(words))
        cleaned_categories.append(category)  # Store the corresponding category
    else:
        print("Skipping invalid description or category:", description, category)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\KIIT\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KIIT\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Skipping invalid description or category: Dissatisfaction with dental office nan
Skipping invalid description or category: Dissatisfaction with Grossmont Dental in providing two different dental service estimates with a $200 difference and charging her over $1000 for a crown. 
 nan
Skipping invalid description or category: Dissatisfaction with having received mail correspondence to his former address although he has requested to change his address on several occasions.  nan
Skipping invalid description or category: Dissatisfaction with Livanta's decision to uphold the skilled nursing facility end his Medicare covered services.  nan
Skipping invalid description or category: Dissatisfaction with mail order being delivered to the incorrect address.  nan
Skipping invalid description or category: Dissatisfaction with not having retina exam at Escondido Premier Eye Care. nan
Skipping invalid description or category: Dissatisfaction with the dental benefit provider regarding the telephone cal

In [10]:
# Remove any remaining NaN or empty string values
cleaned_texts = [text for text in cleaned_texts if text]
cleaned_categories = [category for category in cleaned_categories if isinstance(category, str)]


In [11]:
# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
tfidf_matrix = tfidf_vectorizer.fit_transform(cleaned_texts)

In [13]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, cleaned_categories, test_size=0.2, random_state=42)



In [14]:
# Convert sparse matrices to dense arrays for imputation
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()

In [15]:
# Handle missing values in dense arrays
imputer = SimpleImputer(strategy='constant', fill_value=0)  # Replace NaN with 0
X_train_imputed = imputer.fit_transform(X_train_dense)
X_test_imputed = imputer.transform(X_test_dense)

In [16]:
# Initialize XGBoost classifier
xgb_classifier = XGBClassifier(n_estimators=100, random_state=42)

In [18]:
# Convert boolean labels to integers (1 for True, 0 for False)
y_train_int = [1 if label == 'True' else 0 for label in y_train]
y_test_int = [1 if label == 'True' else 0 for label in y_test]

# Initialize XGBoost classifier
xgb_classifier = XGBClassifier(n_estimators=100, random_state=42)

# Train the classifier
xgb_classifier.fit(X_train_imputed, y_train_int)  # Use y_train_int instead of y_train

# Predict using the classifier
xgb_predictions = xgb_classifier.predict(X_test_imputed)

# Calculate accuracy
xgb_accuracy = accuracy_score(y_test_int, xgb_predictions)  # Use y_test_int instead of y_test

print("XGBoost Accuracy:", xgb_accuracy)


XGBoost Accuracy: 1.0


In [19]:
# Calculate accuracy and convert to percentage
xgb_accuracy = accuracy_score(y_test_int, xgb_predictions) * 100

print("XGBoost Accuracy:", xgb_accuracy, "%")


XGBoost Accuracy: 100.0 %
